# Code to classify images using a Convolutional Neural Network using keras

Code is a combination of stuff from https://github.com/adventuresinML/adventures-in-ml-code/blob/master/keras_cnn.py , Wahid's tutorial and my ice-cube 3d Cnn code.

March 23, 2019

In [1]:
import datetime
print('Notebook last updated: {:%Y-%m-%d %H:%M:%S}'.format(datetime.datetime.now()))

Notebook last updated: 2019-03-23 12:42:23


In [2]:
import numpy as np
import os
import subprocess as sp


import glob
import pickle
import time
import argparse 
import datetime

import matplotlib.pyplot as plt

In [3]:
%matplotlib widget

In [4]:
import tensorflow.keras as keras

from keras import layers, models, optimizers, callbacks  # or tensorflow.keras as keras
# import tensorflow as tf
from sklearn.utils import shuffle
from sklearn.metrics import roc_curve, auc, roc_auc_score
from keras.models import load_model
from keras.datasets import mnist


Using TensorFlow backend.


### Load data

In [5]:

def f_load_data(num_classes):
    '''
    Load the mnist data and process it
    Steps:
    
    '''
    # input image dimensions
    img_x, img_y = 28, 28

    # load the MNIST data set, which already splits into train and test sets for us
    (x_train, y_train), (x_test, y_test) = mnist.load_data()

    # reshape the data into a 4D tensor - (sample_number, x_img_size, y_img_size, num_channels)
    # because the MNIST is greyscale, we only have a single channel - RGB colour images would have 3
    x_train = x_train.reshape(x_train.shape[0], img_x, img_y, 1)
    x_test = x_test.reshape(x_test.shape[0], img_x, img_y, 1)
    input_shape = (img_x, img_y, 1)

    # convert the data to the right type
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices - this is for use in the categorical_crossentropy loss below
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)

    return (x_train, y_train), (x_test, y_test)



### Build model

In [6]:

def f_build_model(inpx,num_classes=10):
    '''
    Build and compile model.
    '''

    inputs = layers.Input(shape=x_train.shape[1:])
    h = inputs

    # Convolutional layers
    conv_sizes=[10,10,10]
    conv_args = dict(kernel_size=(3, 3), activation='relu', padding='same')
    for conv_size in conv_sizes:
        h = layers.Conv2D(conv_size, **conv_args)(h)
        h = layers.MaxPooling2D(pool_size=(2, 2))(h)
    h = layers.Flatten()(h)

    # Fully connected  layers
    h = layers.Dense(64, activation='relu')(h)

    # Ouptut layer
    outputs = layers.Dense(num_classes, activation='sigmoid')(h)

    learn_rate=0.0005

    model = models.Model(inputs, outputs)
    #### change loss function for non-resnet models since 'sparse_categorical_crossentropy' throws up an error.
    opt,loss_fn=optimizers.Adam(lr=learn_rate),'binary_crossentropy'

    model.compile(optimizer=opt, loss=loss_fn, metrics=['accuracy'])

    return model



### Fit model

In [29]:
def f_train_model(model,inpx,inpy,num_epochs,batch_size,verbose,model_save_name):
    '''
    Train model. Returns just history.history
    '''
    cv_fraction=0.33 # Fraction of data for cross validation
    
    history=model.fit(x=x_train, y=y_train,
                    batch_size=batch_size,
                    epochs=num_epochs,
                    verbose=verbose,
                    callbacks = [callbacks.EarlyStopping(monitor='val_loss', min_delta=0,patience=10, verbose=1, mode='auto'),
                                 callbacks.ModelCheckpoint(model_save_name, save_best_only=True, monitor='val_loss', mode='auto') ],
                    validation_split=cv_fraction,
                    shuffle=True
                )
    
    print("Number of parameters",model.count_params())
    
    return history.history

### Test model

In [30]:
# y_pred=model.predict(xdata,verbose=1)

### Plot training and results

In [41]:

    

def f_plot_learning(history):
    '''
    Plot learning curves : Accuracy and Validation
    '''
    
    fig=plt.figure()
    # Plot training & validation accuracy values
    fig.add_subplot(2,1,1)
    xlim=len(history['acc'])
    
    plt.plot(history['acc'],label='Train',marker='o')
    plt.plot(history['val_acc'],label='Validation',marker='*')
#     plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xticks(np.arange(0,xlim,2))
    
    # Plot loss values
    fig.add_subplot(2,1,2)
    plt.plot(history['loss'],label='Train',marker='o')
    plt.plot(history['val_loss'],label='Validation',marker='*')
#     plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.xticks(np.arange(0,xlim,2))

    plt.legend(loc='best')
    

def f_plot_roc_curve(fpr,tpr):
    '''
    Module for roc plot and printing AUC
    '''
    
    plt.figure()
    plt.scatter(fpr,tpr)
    plt.xscale('log')

    plt.xlim([10**-7,1.0])
    plt.ylim([0,1.0])
    # y=x line for comparison
    x=np.linspace(0,1,num=500)
    plt.plot(x,x)
    
    plt.show()

    # AUC 
    auc_val = auc(fpr, tpr)
    print("AUC: ",auc_val)

    return auc_val

### Main function

In [ ]:
if __name__=='__main__':
    
    batch_size = 128
    num_classes = 10
    num_epochs = 100
    verbose=1
    model_save_name='.mdl_weights.h5'
    
    (x_train, y_train), (x_test, y_test)=f_load_data(num_classes)
    model=f_build_model(x_train,num_classes)
    history=f_train_model(model,x_train,x_test,num_epochs,batch_size,verbose,model_save_name)
    model = load_model(model_save_name)
    y_pred=model.predict(x_test,verbose=1)


In [ ]:
if __name__=='__main__':
    ### Plotting learning curve
    f_plot_learning(history)
    
    ### Roc curve
    ##### Since is a case of multi-class classification, for a simple roc curve, just use 1st column for each sample.
    y_pred_1d=y_pred[:,[0]]
    y_test_1d=y_test[:,[0]]
    fpr,tpr,threshold=roc_curve(y_test_1d,y_pred_1d)
    f_plot_roc_curve(fpr,tpr)

### To do : 
- Roc curve for multi-class